## Feasibility of Student Accomodation ##

### Description ###

Universities cannot provide residences for all their students.  Student Accomodation in South Africa has become extremely profitable and competitive. 

Choosing the correct location for your student accomodation is critical to ensure that you attract students wanting to rent.

Factors affecting decision of students such as:

* Distance from university
* Proximity of Food and Drink Shops
* Proximity of Restaurants 
* Proximity of Recreational Activities
* Proximity of NIght Life

can be used to determine if a location will be feasible.

### Data ###

Using the Four Square data, and given the geo-location of where the client would like to purchase student accomodation, we will retrieve the number of Food and Drink Shops, Restaurants, Recreational Activities and Night Life within 2000m from the location.  These will then be scored according to the following table:


| Category           |  Score       |    
| -------------      |:-------------:| 
| Food and Drink     | 5             | 
| Restaurants        | 2             | 
| Recreation         | 1      |  
| Night Life         | 1             

Distance to the university will also be scored, with 100 points being given for within 100m and then decreasing by 5 points for every 100m away)


The score will give an indication of feasibility of the location.

The client has chosen 5 possible locations, and we will compare each one to see who has the highest score.

### Methodology ###

First had to determine what factors influence students decisions, so I interviewed 10 students who stayed in Student Accomodation, as well as 3 Landlords.

This gave me what variables to use, and what they weight they have on overall decisions.

We also have to calculate the distance of the planned accomodation from the University, and then score that.

I did 4 separate API calls for each category and then used the total of each call to apply to the scoring table.

We then add all the various values to get an overall score, and compare for each location chosen by the client.

In [15]:
import pandas as pd
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import haversine as hs
from haversine import Unit

import getpass
CLIENT_ID = getpass.getpass()
CLIENT_SECRET = getpass.getpass()


def calc_distance_score(distance):
    if distance <= 100:
          return 100
    else:
          distance -= 105
          moddistance = distance//100
          value = 100-(moddistance*5)
          if value > 0:
             return value
          return 0
    
    
 
VERSION = '20180605' # Foursquare API version
score_arr=[0,0,0,0,0]
houses_lat=[-33.98981131481847,-33.96134038835347,-33.89582193295607,-33.91717915865144, -33.92607105991247]
houses_long=[18.465643577626018,18.51576869742209,18.49845269784273,18.39551998148739,18.45133701501232]

for index,neighborhood_latitude in enumerate(houses_lat):
      neighborhood_longitude = houses_long[index]
        
#neighborhood_longitude=18.470782183145282
#neighborhood_latitude=-33.96792722561728
      LIMIT = 100 # limit of number of venues returned by Foursquare API
      radius = 2000 # define radius
      urls=[]
      categories = ['4d4b7105d754a06374d81259','4d4b7105d754a06376d81259','4d4b7105d754a06377d81259','4bf58dd8d48988d1f9941735']
      for category in categories: 
            urls.append('https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT,
            category))
            results=[]
            for url in urls:
                results.append(requests.get(url).json())
      venues1 = results[0]['response']['groups'][0]['items'] 
      venues2 = results[1]['response']['groups'][0]['items'] 
      venues3 = results[2]['response']['groups'][0]['items'] 
      venues4 = results[3]['response']['groups'][0]['items'] 

      loc1=(neighborhood_latitude,neighborhood_longitude)
      loc2=(-33.95771905704446,18.461027204592227)
      distance_gap= hs.haversine(loc1,loc2,unit=Unit.METERS)
      distance_score=calc_distance_score(distance_gap)


      print('The number of Restaurants within your vincinity : '+str(len(venues1)))
      print('The number of Night Life Spots within your vincinity : '+str(len(venues2)))
      print('The number of Other Recreational Activities within your vincinity : '+str(len(venues3)))
      print('The number of Food and Drink Shops within your vincinity : '+str(len(venues4)))
      print('The distance to the university (in meters) is : '+str(distance_gap))



      score = distance_score

      score += len(venues1)*2+len(venues2)*1+len(venues3)*1+len(venues4)*5

      print('The score for your choice is: '+str(score))
      score_arr[index]=score

print('Results')
score_arr

········
········
The number of Restaurants within your vincinity : 79
The number of Night Life Spots within your vincinity : 16
The number of Other Recreational Activities within your vincinity : 20
The number of Food and Drink Shops within your vincinity : 13
The distance to the university (in meters) is : 3593.8021238586143
The score for your choice is: 259
The number of Restaurants within your vincinity : 13
The number of Night Life Spots within your vincinity : 3
The number of Other Recreational Activities within your vincinity : 3
The number of Food and Drink Shops within your vincinity : 2
The distance to the university (in meters) is : 5064.774582204053
The score for your choice is: 42
The number of Restaurants within your vincinity : 45
The number of Night Life Spots within your vincinity : 5
The number of Other Recreational Activities within your vincinity : 11
The number of Food and Drink Shops within your vincinity : 13
The distance to the university (in meters) is : 7700.2

[259, 42, 171, 406, 154]

In [14]:
# display results

import folium
map_cape_town = folium.Map(location=[-33.95771905704446, 18.461027204592227], zoom_start=10)
for index,neighborhood_latitude in enumerate(houses_lat):
      neighborhood_longitude = houses_long[index]
      folium.CircleMarker(
        [neighborhood_latitude, neighborhood_longitude],
        radius=score_arr[index]/10,
        popup='House '+str(index+1),
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cape_town)    
folium.CircleMarker(
        [-33.95771905704446, 18.461027204592227],
        radius=5,
        popup='University',
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cape_town) 
map_cape_town

### Results ####

As the results show, the best place to have student accomodation is House 4, which has the best score.  It is displayed on the map by the largest circle.

From the results we can show the following:

House 1

| Category           |  Number |    
| -------------      |-------------:| 
| Food and Drink     | 13             | 
| Restaurants        | 79             | 
| Recreation         | 20      |  
| Night Life         | 16             


House 2

| Category           |  Number |    
| -------------      |-------------:| 
| Food and Drink     | 2             | 
| Restaurants        | 13             | 
| Recreation         | 3     |  
| Night Life         | 3             

House 3

| Category           |  Number |    
| -------------      |-------------:| 
| Food and Drink     | 13             | 
| Restaurants        | 45             | 
| Recreation         | 5      |  
| Night Life         | 1             


House 4

| Category           |  Number |    
| -------------      |-------------:| 
| Food and Drink     | 25             | 
| Restaurants        | 100             | 
| Recreation         | 28      |  
| Night Life         | 16             



House 5

| Category           |  Number |    
| -------------      |-------------:| 
| Food and Drink     | 11             | 
| Restaurants        | 42             | 
| Recreation         | 7      |  
| Night Life         | 8             


Distance from University


| House            |   Distance(m) |    
| -------------      |-------------:| 
| House 1     | 3593.8             | 
| House 2        | 5064.8             | 
| House 3         | 7700.3      |  
| House 4         | 7539.3  
| House 5        | 3630.9  

### Discussion ###


The results should also take into context of house and rental prices, as accomodation in the 'better' areas might be more expensive.

The locations closer to the city had a higher score. Traffic in the city is something to consider, however with universities moving into a more remote/blended learning mode of teaching, this might not be a large factor for the future.


### Conclusion ###

Visualising the results on a map make it easy for a would-be landlord to identify good locations for student accommodation.  The weight of the factors identified can also be adjusted depending on the target market, and additional factors could also be added such as crime rate, or even average cost of housing in the area.

House 4 seemed to be the best location for the client.



